# Loading libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

## Scenario

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

## Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

Here is the list of steps to be followed (building a simple model without balancing the data):

- Import the required libraries and modules that you would need.
- Read that data into Python and call the dataframe churnData.
- Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.
- Check for null values in the dataframe. Replace the null values.
- Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
- Scale the features either by using normalizer or a standard scaler.
- Split the data into a training set and a test set.
- Fit a logistic Regression model on the training data.
- Fit a Knn Classifier model on the training data

## Data splitting

### Loading a dataset for classification

In [ ]:
# read data
churnData = pd.read_csv("DATA_Customer-Churn.csv")

In [ ]:
#check data types for all columns
churnData.dtypes

In [ ]:
churnData

In [ ]:
#convert "TotalCharges" to numeric using pd.to_numeric

pd.get_dummies(churnData["Contract"])

churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors='coerce')
churnData.dtypes

#check for and replace null values 
churnData.dropna(inplace=True)

#scale features by using normalizer or a standard scaler:  tenure, SeniorCitizen, MonthlyCharges and TotalCharges
#churnData = churnData[["tenure","SeniorCitizen","MonthlyCharges","TotalCharges","Churn"]]

#convert values to 0 and 1
def x_y_to_1_0(value:[str]=None, x="Yes", y="No"):
    if value == x:
        return 1
    elif value == y:
        return 0
    else:
        return None
    
churnData["Churn"] = list(map(x_y_to_1_0, churnData["Churn"]))
churnData["Partner"] = list(map(x_y_to_1_0, churnData["Partner"]))
churnData["Dependents"] = list(map(x_y_to_1_0, churnData["Dependents"]))
churnData["PhoneService"] = list(map(x_y_to_1_0, churnData["PhoneService"]))
churnData["OnlineSecurity"] = list(map(x_y_to_1_0, churnData["OnlineSecurity"]))
churnData["OnlineBackup"] = list(map(x_y_to_1_0, churnData["OnlineBackup"]))
churnData["DeviceProtection"] = list(map(x_y_to_1_0, churnData["DeviceProtection"]))
churnData["TechSupport"] = list(map(x_y_to_1_0, churnData["TechSupport"]))
churnData["StreamingTV"] = list(map(x_y_to_1_0, churnData["StreamingTV"]))
churnData["StreamingMovies"] = list(map(x_y_to_1_0, churnData["StreamingMovies"]))

def gender_conv(value:[str]=None, x="Female", y="Male"):
    if value == x:
        return 1
    elif value == y:
        return 0
    else:
        return None

churnData["gender"] = list(map(gender_conv, churnData["gender"]))

#def contract_conv(value:[str]=None, x="Month-to-month", y="One year"):
    #if value == x:
        #return 1
    #elif value == y:
      #  return 0
   # else:
        #return 2

#churnData["Contract"] = list(map(contract_conv, churnData["Contract"]))





In [ ]:
categoricals_features= churnData.select_dtypes("object")
cat_cols=pd.get_dummies(categoricals_features, drop_first=True)

In [ ]:
churnData.dropna(inplace=True)

#X/Y split
X=churnData.drop("Churn", axis=1)
y=churnData["Churn"]

In [ ]:
X = X._get_numeric_data()
X = pd.concat([X,cat_cols],axis=1)

In [ ]:
X

# Modelling

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix

log_model = LogisticRegression() 

#Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)


trans = PowerTransformer()

trans.fit(X_train)

X_train_mod = trans.transform(X_train)
X_test_mod  = trans.transform(X_test)

log_model.fit(X_train_mod, y_train)

y_pred_train_log = log_model.predict(X_train_mod)
y_pred_test_log = log_model.predict(X_test_mod)

performance_log = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, y_pred_train_log),
                                         precision_score(y_train, y_pred_train_log,pos_label="Yes"),
                                         recall_score(y_train, y_pred_train_log,pos_label="Yes")],
                               'Test': [accuracy_score(y_test, y_pred_test_log),
                                        precision_score(y_test, y_pred_test_log,pos_label="Yes"),
                                        recall_score(y_test, y_pred_test_log,pos_label="Yes")]})

display(performance_log)

print("Confusion matrix for the train set")
print(confusion_matrix(y_train,y_pred_train_log))
plot_confusion_matrix(log_model,X_train_mod,y_train, values_format = 'd')
plt.show()

print()
print()

print("Confusion matrix for the test set")
print(confusion_matrix(y_test, y_pred_test_log))
plot_confusion_matrix(log_model,X_test_mod,y_test, values_format = 'd')
plt.show()

## KNN Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=3)


#Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11)


trans = PowerTransformer()

trans.fit(X_train)

X_train_mod = trans.transform(X_train)
X_test_mod  = trans.transform(X_test)

knn_model.fit(X_train_mod, y_train)

y_pred_train_log = knn_model.predict(X_train_mod)
y_pred_test_log = knn_model.predict(X_test_mod)

performance_log = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, y_pred_train_log),
                                         precision_score(y_train, y_pred_train_log,pos_label="Yes"),
                                         recall_score(y_train, y_pred_train_log,pos_label="Yes")],
                               'Test': [accuracy_score(y_test, y_pred_test_log),
                                        precision_score(y_test, y_pred_test_log,pos_label="Yes"),
                                        recall_score(y_test, y_pred_test_log,pos_label="Yes")]})

display(performance_log)

print("Confusion matrix for the train set")
print(confusion_matrix(y_train,y_pred_train_log))
plot_confusion_matrix(knn_model,X_train_mod,y_train, values_format = 'd')
plt.show()

print()
print()

print("Confusion matrix for the test set")
print(confusion_matrix(y_test, y_pred_test_log))
plot_confusion_matrix(knn_model,X_test_mod,y_test, values_format = 'd')
plt.show()

- Fit a Decision Tree Classifier on the training data.
- Check the accuracy on the test data.

## Decision Tree

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=11)

# Bear in mind that sklearn uses a different function for decision trees used for 
# classification ( to predict a categorical feature ): DecisionTreeClassifier() 
dt = DecisionTreeClassifier(max_depth=3)

dt.fit(X_train, y_train)

y_pred_train_dt = dt.predict(X_train)
y_pred_test_dt = dt.predict(X_test)


performance_df = pd.DataFrame({'Error_metric': ['Accuracy','Precision','Recall'],
                               'Train': [accuracy_score(y_train, y_pred_train_dt),
                                         precision_score(y_train, y_pred_train_dt, pos_label="Yes"),
                                         recall_score(y_train, y_pred_train_dt, pos_label="Yes")],
                               'Test': [accuracy_score(y_test, y_pred_test_dt),
                                        precision_score(y_test, y_pred_test_dt, pos_label="Yes"),
                                        recall_score(y_test, y_pred_test_dt, pos_label="Yes")]})

display(performance_df)

print("Confusion matrix for the train set")
print(confusion_matrix(y_train,y_pred_train_dt).T)
plot_confusion_matrix(dt,X_train,y_train, values_format = 'd')
plt.show()

print()
print()

print("Confusion matrix for the test set")
print(confusion_matrix(y_test,y_pred_test_dt).T)
plot_confusion_matrix(dt,X_test,y_test, values_format = 'd')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (34,20))

plot_tree(dt,filled = True, rounded=True,feature_names=X.columns)
plt.show() 